<a href="https://colab.research.google.com/github/MudassirABBASSi/Computer-Vision/blob/main/paper_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import zipfile

# Unzipping the dataset
zip_ref = zipfile.ZipFile('/content/OMR Answear sheet.v2i.coco.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [13]:
import os

def count_images(directory):
    """Counts the number of image files in a directory and its subdirectories."""
    image_count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_count += 1
    return image_count

# Define paths to images
train_image_count = count_images('/content/test')
val_image_count = count_images('/content/valid')

print(f"Number of training images: {train_image_count}")
print(f"Number of validation images: {val_image_count}")


Number of training images: 10
Number of validation images: 10


In [15]:
# Install Detectron2 (only needed if not installed)
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Import required libraries
import os
import cv2
import json
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ktmlmfo4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ktmlmfo4
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done


In [17]:
# Define dataset paths
dataset_root = "/content/OMR_Answear_sheet.v2i.coco"

train_images = os.path.join(dataset_root, "images", "train")
val_images   = os.path.join(dataset_root, "images", "val")
train_annotations = os.path.join(dataset_root, "annotations", "instances_train.json")
val_annotations   = os.path.join(dataset_root, "annotations", "instances_val.json")

# Register training and validation datasets
register_coco_instances("omr_train", {}, train_annotations, train_images)
register_coco_instances("omr_val", {}, val_annotations, val_images)

print("COCO dataset registered successfully!")


COCO dataset registered successfully!


# testing

In [14]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

dataset_dicts = detectron2.data.DatasetCatalog.get("omr_train")
metadata = MetadataCatalog.get("omr_train")

# Load and visualize one sample image
sample_data = dataset_dicts[0]
image_path = sample_data["file_name"]

# Check if the image file exists
if os.path.exists(image_path):
    image = cv2.imread(image_path)
    visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.0)
    output = visualizer.draw_dataset_dict(sample_data)
    # Save the visualization
    cv2.imwrite("dataset_sample.jpg", output.get_image()[:, :, ::-1])
    print("Sample dataset image saved as dataset_sample.jpg")
else:
    print(f"Error: Image file not found at {image_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/OMR_Answear_sheet.v2i.coco/annotations/instances_train.json'

In [ ]:
cfg = get_cfg()

# Load Faster R-CNN model with ResNet-50 backbone
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Set dataset names
cfg.DATASETS.TRAIN = ("omr_train",)
cfg.DATASETS.TEST = ("omr_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Use pretrained weights from COCO
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# Training parameters
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Learning rate
cfg.SOLVER.MAX_ITER = 3000   # Number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 16  # Update based on your dataset classes

# Create output directory if not exists
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

print("Configuration setup completed.")


In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold
predictor = DefaultPredictor(cfg)

# Load an image from the validation set for testing
test_image_path = os.path.join(val_images, os.listdir(val_images)[0])
image = cv2.imread(test_image_path)

# Get predictions
outputs = predictor(image)

# Visualize results
visualizer = Visualizer(image[:, :, ::-1], metadata=MetadataCatalog.get("omr_val"), scale=1.0)
output = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))

# Save inference output
cv2.imwrite("inference_result.jpg", output.get_image()[:, :, ::-1])
print("Inference result saved as inference_result.jpg")
